In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib notebook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nettools.multiplex import InterMeasures
from nettools.monoplex import CentralityMeasure
from nettools.epidemic import SISMultiplex, SIRMultiplex
from nettools.utils import load_multinet_by_name, NX_CENTRALITY

# Centrality

After analysing interdependence for multilayer networks, I decided to measure centrality for nodes in purpose of finding the *best spreader* in network. In my reasearch I will test few commonly used metrics. <br><br>
**Metrics**
* Degree
* Closeness
* Betweenness
* Eigenvector
* Pagerank
* Hits
* K-shell
* Voterank

After finding most central node in network, This node become infected, and start *disease / information spreading* on network.

# Load Datasets

In reaserch same as before I use *London Transportation*, *European Airplanes* and *FAO Trade Network*.

In [3]:
network_london = load_multinet_by_name('london')

In [4]:
network_fao = load_multinet_by_name('fao')

In [5]:
network_euair = load_multinet_by_name('EUAir')

# Construct network attributes dictionary
**InterMeasures** object accept dictionary with network parameters, but
it's possible to pass database name instead.

### *European Airlines* Network properties

In [6]:
network_attrs_eu = {}
network_attrs_eu['loaded_network'] = network_euair[0]
network_attrs_eu['network_graph_np'] = network_euair[1]
network_attrs_eu['network_weights_np'] = network_euair[2]
network_attrs_eu['mapping'] = network_euair[3]
network_attrs_eu['layers_attr'] = network_euair[4]

### *London Transportation* Network properties

In [7]:
network_attrs_lt = {}
network_attrs_lt['loaded_network'] = network_london[0]
network_attrs_lt['network_graph_np'] = network_london[1]
network_attrs_lt['network_weights_np'] = network_london[2]
network_attrs_lt['mapping'] = network_london[3]
network_attrs_lt['layers_attr'] = network_london[4]

### *FAO Trade Network* Network properties

In [8]:
network_attrs_fao = {}
network_attrs_fao['loaded_network'] = network_fao[0]
network_attrs_fao['network_graph_np'] = network_fao[1]
network_attrs_fao['network_weights_np'] = network_fao[2]
network_attrs_fao['mapping'] = network_fao[3]
network_attrs_fao['layers_attr'] = network_fao[4]

# Reasearch approach
In this notebook I going to measure *centrality* for three networks. Each network was previously examined in terms of *interdependence*.<br>
**Experiment steps**
* Aggregate all layers and measure centrality
* Aggregate layers based on interdependence
* Multilayer methods (eg. *KS Index*)
* Proposal:
    * Best optimal spreader for each layer
    * Optimalization algorithm, find best coefficients **b**.

In [9]:
im_eu = InterMeasures(network_attr=network_attrs_eu)
im_lt = InterMeasures(network_attr=network_attrs_lt)
im_fao = InterMeasures(network_attr=network_attrs_fao)

# Aggregate networks
First I will examine centrality for aggregated network. In this case network is aggregated using equal weights.<br>
**y = w1 \* layer1 + w2 \* layer2, w1 = w2**

In [10]:
agg_net_eu = im_eu.agg_net
agg_net_lt = im_lt.agg_net
agg_net_fao = im_fao.agg_net

Find central ten central nodes using different methods for monoplex networks

In [11]:
cent_dict = {}
cm_eu = CentralityMeasure(agg_net_eu)
for method in NX_CENTRALITY.keys():
    results = cm_eu.network_cn(method)
    if method == 'hits':
        results = results[1]
    best_10 = sorted(results.items(), key=lambda x: x[1])[::-1][:10]
    cent_dict[method] = [cnt[0] for cnt in best_10]
pd.DataFrame.from_dict(cent_dict)

,betweenness,closeness,degree,eigenvector,hits,k-shell,pagerank,voterank
0,30,14,11,14,14,251,14,14
1,13,37,14,49,49,214,13,49
2,23,1,37,82,82,198,49,37
3,11,39,251,63,63,180,39,39
4,14,165,49,39,39,165,37,1
5,251,21,39,37,37,121,11,251
6,39,49,1,6,6,107,23,63
7,76,6,107,1,1,100,63,11
8,37,63,23,165,165,95,251,6
9,27,82,82,65,65,82,1,82


In [20]:
agg_net_eu.shape
sis = SISMultiplex(im_eu.get_network_adjacency(), beta=0.3, mu=0.01, inter_beta=1.0, 
                   inter_rec=0.01, seed_nodes=[cent_dict['closeness'][0]])
sis.run(epochs=200, visualize=False)
sis.get_num('i')

15273